In [36]:
import pandas as pd

In [58]:
from google.cloud import bigquery

bqclient = bigquery.Client()
#Nom_base_fran__ais, Total_poste_non_d__compos__, Code_de_la_cat__gorie
# Download query results.
query_string = """
SELECT *
FROM `hackathon-climat-01.hackathon_bq.adame_base_carbon`
WHERE (Statut_de_l___l__ment="Valide générique" OR Statut_de_l___l__ment="Valide spécifique") 
"""

df = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(
        # Optionally, explicitly request to use the BigQuery Storage API. As of
        # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
        # API is used by default.
        create_bqstorage_client=True,
    )
)
print(df.shape)
df.head(2)

(9995, 68)


,string_field_0,Type_Ligne,Identifiant_de_l___l__ment,Structure,Type_de_l___l__ment,Statut_de_l___l__ment,Nom_base_fran__ais,Nom_base_anglais,Nom_base_espagnol,Nom_attribut_fran__ais,...,Code_gaz_suppl__mentaire_2,Valeur_gaz_suppl__mentaire_2,Code_gaz_suppl__mentaire_3,Valeur_gaz_suppl__mentaire_3,Code_gaz_suppl__mentaire_4,Valeur_gaz_suppl__mentaire_4,Code_gaz_suppl__mentaire_5,Valeur_gaz_suppl__mentaire_5,Autres_GES,CO2b
0,0,Elément,34052,élément non décomposé,Facteur d'émission,Valide générique,"""\tSalade César au poulet (salade verte""",Caesar's salad (salad,None,"fromage, croûtos, sauce)",...,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN
1,1,Elément,33558,élément non décomposé,Facteur d'émission,Valide générique,"""Brioche fourrée crème pâtissière (type """"chin...",Brioche,None,préemballée,...,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN


## Preprocessing
- we select the wanted products : achats de biens
- to lower case
- remove accents

In [63]:
# Select the good category
df_agro = df[df.loc[:, 'Code_de_la_cat__gorie'].apply(lambda x: 'Achats de biens > Produits agro-alimentaires' in x)]

# Remove upper cases
df_agro.loc[:, "Nom_base_fran__ais"] = df_agro.loc[:, "Nom_base_fran__ais"].apply(lambda x: x.lower())

# Remove some characters
df_agro.loc[:, "Nom_base_fran__ais"] = df_agro.loc[:, "Nom_base_fran__ais"].apply(lambda x: x.replace('"', ""))
                                                                                  
print(df_agro.shape)
df_agro.head(2)

(1869, 68)


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,string_field_0,Type_Ligne,Identifiant_de_l___l__ment,Structure,Type_de_l___l__ment,Statut_de_l___l__ment,Nom_base_fran__ais,Nom_base_anglais,Nom_base_espagnol,Nom_attribut_fran__ais,...,Code_gaz_suppl__mentaire_2,Valeur_gaz_suppl__mentaire_2,Code_gaz_suppl__mentaire_3,Valeur_gaz_suppl__mentaire_3,Code_gaz_suppl__mentaire_4,Valeur_gaz_suppl__mentaire_4,Code_gaz_suppl__mentaire_5,Valeur_gaz_suppl__mentaire_5,Autres_GES,CO2b
0,0,Elément,34052,élément non décomposé,Facteur d'émission,Valide générique,\tsalade césar au poulet (salade verte,Caesar's salad (salad,None,"fromage, croûtos, sauce)",...,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN
1,1,Elément,33558,élément non décomposé,Facteur d'émission,Valide générique,brioche fourrée crème pâtissière (type chinois),Brioche,None,préemballée,...,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN


In [64]:
import unidecode
df_agro.Nom_base_fran__ais = df_agro.Nom_base_fran__ais.apply(lambda recette: unidecode.unidecode(recette.strip()))
df_agro.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,string_field_0,Type_Ligne,Identifiant_de_l___l__ment,Structure,Type_de_l___l__ment,Statut_de_l___l__ment,Nom_base_fran__ais,Nom_base_anglais,Nom_base_espagnol,Nom_attribut_fran__ais,...,Code_gaz_suppl__mentaire_2,Valeur_gaz_suppl__mentaire_2,Code_gaz_suppl__mentaire_3,Valeur_gaz_suppl__mentaire_3,Code_gaz_suppl__mentaire_4,Valeur_gaz_suppl__mentaire_4,Code_gaz_suppl__mentaire_5,Valeur_gaz_suppl__mentaire_5,Autres_GES,CO2b
0,0,Elément,34052,élément non décomposé,Facteur d'émission,Valide générique,salade cesar au poulet (salade verte,Caesar's salad (salad,None,"fromage, croûtos, sauce)",...,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN
1,1,Elément,33558,élément non décomposé,Facteur d'émission,Valide générique,brioche fourree creme patissiere (type chinois),Brioche,None,préemballée,...,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN
2,2,Elément,32682,élément non décomposé,Facteur d'émission,Valide générique,gouter sec fourre (sandwiche) parfum chocolat,Biscuit (cookie),None,None,...,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN
3,3,Elément,32683,élément non décomposé,Facteur d'émission,Valide générique,gouter sec fourre (sandwiche) parfum fruits,Biscuit (cookie),None,None,...,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN
4,4,Elément,32681,élément non décomposé,Facteur d'émission,Valide générique,gouter sec fourre (sandwiche) parfum lait ou v...,Biscuit (cookie),None,None,...,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN


In [65]:
# Remove all inside brackets
import re 
df2 = df_agro.copy()
df2.Nom_base_fran__ais = df2.Nom_base_fran__ais.apply(lambda x: re.sub("[\(\[].*?[\)\]]", "", x))

# Remove some characters
df2.Nom_base_fran__ais = df2.Nom_base_fran__ais.apply(lambda x: x.replace("%", "").replace("_", "").replace(".", ""))

df2 = df2.drop(['string_field_0'], axis=1)
print(df2.shape)
df2.head(2)

(1869, 67)


,Type_Ligne,Identifiant_de_l___l__ment,Structure,Type_de_l___l__ment,Statut_de_l___l__ment,Nom_base_fran__ais,Nom_base_anglais,Nom_base_espagnol,Nom_attribut_fran__ais,Nom_attribut_anglais,...,Code_gaz_suppl__mentaire_2,Valeur_gaz_suppl__mentaire_2,Code_gaz_suppl__mentaire_3,Valeur_gaz_suppl__mentaire_3,Code_gaz_suppl__mentaire_4,Valeur_gaz_suppl__mentaire_4,Code_gaz_suppl__mentaire_5,Valeur_gaz_suppl__mentaire_5,Autres_GES,CO2b
0,Elément,34052,élément non décomposé,Facteur d'émission,Valide générique,salade cesar au poulet (salade verte,Caesar's salad (salad,None,"fromage, croûtos, sauce)","chicken, croûtons, sauce)",...,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN
1,Elément,33558,élément non décomposé,Facteur d'émission,Valide générique,brioche fourree creme patissiere,Brioche,None,préemballée,"filled with custard (Chinese brioche type), pr...",...,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN


In [66]:
df2.to_csv('data_base_carbon.csv', header=True, index=False)